In [27]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.4.0
Nibabel version: 5.3.2
scikit-image version: 0.24.0
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: 2.18.0
gdown version: 5.2.0
TorchVision version: 0.20.1+cu124
tqdm version: 4.66.6
lmdb version: 1.5.1
psutil version: 6.1.0
pandas version: 2.2.3
einops version: 0.8.0
transformers version: 4.46.2
mlflow version: 2.17.2
pynrrd version: 1.1.1
clearml version: 1.16.5

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [28]:
from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [29]:
logger.remove()

In [30]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
curr_dir = msmri_home / "analysis" / "choroid_pineal_pituitary_crosstrain"
training_work_dirs = msmri_home / "training_work_dirs"
modalities = ["flair", "t1"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]

### Second Set

In [31]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary3-2"
work_dir = training_work_dirs / work_dir_name

In [32]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset2.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 19562.99it/s]


In [33]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [34]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [35]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-07 16:37:58,698 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2
2025-01-07 16:37:58,700 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/datalist.json.
2025-01-07 16:37:58,700 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/datalist.json.
2025-01-07 16:37:58,711 - INFO - Using user defined command running prefix , will override other settings


In [36]:
runner.run()

2025-01-07 16:37:58,715 - INFO - Running data analysis...
2025-01-07 16:37:58,716 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:29<00:00,  1.03it/s]

2025-01-07 16:38:27,917 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-07 16:38:27,917 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/datastats.yaml.
2025-01-07 16:38:27,924 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/datastats_by_case.yaml, this may take a while.
2025-01-07 16:38:28,061 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz



algo_templates.tar.gz: 104kB [00:00, 518kB/s]                              

2025-01-07 16:38:28,269 - INFO - Downloaded: /tmp/tmp6pyldrtq/algo_templates.tar.gz
2025-01-07 16:38:28,269 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmp6pyldrtq/algo_templates.tar.gz.
2025-01-07 16:38:28,270 - INFO - Writing into directory: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2.
2025-01-07 16:38:28,320 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/swinunetr_0
2025-01-07 16:38:28,347 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/swinunetr_1
2025-01-07 16:38:28,373 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/swinunetr_2
2025-01-07 16:38:28,399 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/swinunetr_3
2025-01-07 16:38:28,425 - INFO - Generated:/home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/swinunetr_4
2025-01-07 16:

...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-07 19:12:48,934 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-07 19:12:48,934 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/07 19:12:49 INFO mlflow.

2025-01-07 19:53:24,766 - INFO - Ensembling using single GPU!
2025-01-07 19:53:24,766 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/ensemble_output will be used to save ensemble predictions.
2025-01-07 19:53:24,767 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/ensemble_output is created to save ensemble predictions
2025-01-07 19:53:24,797 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-07 19:53:24,798 - INFO - swinunetr_0
2025-01-07 19:53:24,798 - INFO - swinunetr_1
2025-01-07 19:53:24,798 - INFO - swinunetr_2
2025-01-07 19:53:24,799 - INFO - swinunetr_3
2025-01-07 19:53:24,799 - INFO - swinunetr_4
2025-01-07 19:53:24,799 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-2/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Third Set

In [37]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary3-3"
work_dir = training_work_dirs / work_dir_name

In [38]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset3.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 107.45it/s]


In [39]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [40]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [41]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-07 20:03:37,946 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3
2025-01-07 20:03:37,947 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/datalist.json.
2025-01-07 20:03:37,947 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/datalist.json.
2025-01-07 20:03:37,947 - INFO - Using user defined command running prefix , will override other settings


In [42]:
runner.run()

2025-01-07 20:03:37,957 - INFO - Running data analysis...


2025-01-07 20:03:37,957 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:26<00:00,  1.13it/s]

2025-01-07 20:04:04,505 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-07 20:04:04,506 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/datastats.yaml.
2025-01-07 20:04:04,511 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/datastats_by_case.yaml, this may take a while.


2025-01-07 20:04:04,687 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-07 22:31:28,470 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-07 22:31:28,470 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/07 22:31:28 INFO mlflow.

2025-01-07 23:16:25,263 - INFO - Ensembling using single GPU!
2025-01-07 23:16:25,264 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/ensemble_output will be used to save ensemble predictions.
2025-01-07 23:16:25,264 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/ensemble_output is created to save ensemble predictions
2025-01-07 23:16:25,289 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-07 23:16:25,290 - INFO - swinunetr_0
2025-01-07 23:16:25,290 - INFO - swinunetr_1
2025-01-07 23:16:25,290 - INFO - swinunetr_2
2025-01-07 23:16:25,290 - INFO - swinunetr_3
2025-01-07 23:16:25,291 - INFO - swinunetr_4
2025-01-07 23:16:25,291 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-3/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

### Fourth Set

In [43]:
dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary3-4"
work_dir = training_work_dirs / work_dir_name

In [44]:
dataset, _ = preprocess.load_dataset(curr_dir / "dataset4.json")
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, ["CH", "SRS", "ED", "DT"])
dataset_proc.prepare_images(modalities)
dataset = dataset_proc.dataset

100%|██████████| 40/40 [00:00<00:00, 104.33it/s]


In [45]:
train_data = []
test_data = []
for scan in dataset:
    if scan.cond == 'tr' and scan.has_label:
        train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
    elif scan.cond == 'ts' and scan.has_label():
        test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


print(f"Train num total: {len(train_data)}")
print(f"Test num: {len(test_data)}")

Train num total: 30
Test num: 10


In [46]:
n_folds = 5
datalist = {
    "testing": test_data,
    "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
}

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# dataroot_dir = "/mnt/h"
# if not os.path.isdir(dataroot_dir):
#     os.makedirs(dataroot_dir)

datalist_file = os.path.join(work_dir, "datalist.json")
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

In [47]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

2025-01-07 23:26:47,921 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4
2025-01-07 23:26:47,922 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/datalist.json.
2025-01-07 23:26:47,922 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/datalist.json.
2025-01-07 23:26:47,923 - INFO - Using user defined command running prefix , will override other settings


In [48]:
runner.run()

2025-01-07 23:26:47,932 - INFO - Running data analysis...


2025-01-07 23:26:47,932 - INFO - Found 1 GPUs for data analyzing!


100%|██████████| 30/30 [00:29<00:00,  1.03it/s]

2025-01-07 23:27:17,204 - INFO - Data spacing is not completely uniform. MONAI transforms may provide unexpected result
2025-01-07 23:27:17,204 - INFO - Writing data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/datastats.yaml.
2025-01-07 23:27:17,210 - INFO - Writing by-case data stats to /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/datastats_by_case.yaml, this may take a while.


2025-01-07 23:27:17,387 - INFO - BundleGen from https://github.com/Project-MONAI/research-contributions/releases/download/algo_templates/e4cf5a1.tar.gz


...

monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().


2025-01-08 02:08:36,288 - INFO - Downloaded: /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt
2025-01-08 02:08:36,289 - INFO - Expected md5 is None, skip md5 check for file /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/swinunetr_4/pretrained_model/swin_unetr.base_5000ep_f48_lr2e-4_pretrained.pt.


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
2025/01/08 02:08:36 INFO mlflow.

2025-01-08 02:47:40,408 - INFO - Ensembling using single GPU!
2025-01-08 02:47:40,409 - INFO - The output_dir is not specified. /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/ensemble_output will be used to save ensemble predictions.
2025-01-08 02:47:40,409 - INFO - Directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/ensemble_output is created to save ensemble predictions
2025-01-08 02:47:40,434 - INFO - Auto3Dseg picked the following networks to ensemble:
2025-01-08 02:47:40,434 - INFO - swinunetr_0
2025-01-08 02:47:40,434 - INFO - swinunetr_1
2025-01-08 02:47:40,434 - INFO - swinunetr_2
2025-01-08 02:47:40,435 - INFO - swinunetr_3
2025-01-08 02:47:40,435 - INFO - swinunetr_4
2025-01-08 02:47:40,435 - INFO - Auto3Dseg ensemble prediction outputs will be saved in /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary3-4/ensemble_output.


Ensembling (rank 0)...:   0%|          | 0/10 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w